<a href="https://colab.research.google.com/github/the1quadfather/nfl_play_predictor/blob/main/playcall_predictor_sgdclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# NFL Playcall Predictor
# Based on down, yard line, yards-to-go, time, quarter, score differential, predict whether the next play is
# a run or a pass
# Play-by-play data: 2013-2023, NFLSavant.com


import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

from google.colab import drive
drive.mount('/content/drive')

# PBP CSV data import
print('Importing files...')
#file_path = '/content/drive/MyDrive/Schraeder Management/Product Dev/NFL Play Predictor/NFL_PBP_combined_playground_csv_no2020.csv'
file_path = '/content/drive/MyDrive/Schraeder Management/Product Dev/NFL Play Predictor/2023.csv'
# Marginal improvement (1%) when evaluating a single year compared to all years
df_x = pd.read_csv(file_path, usecols=["SeasonYear", "Quarter", "Minute",
                                                                   "Second", "Down", "ToGo", "YardLine", "OFF", "DEF"])
df_x.head()
df_y = pd.read_csv(file_path, usecols=["Rush/Pass/Special"])

df_x.dropna(inplace=True)
df_y.dropna(inplace=True) # Remember that inplace edits the current data file, it doesn't create an edited copy

print('Import complete!')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Importing files...
Import complete!


In [8]:
# Perform multi-class classification using SGD Classifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X = df_x
Y = np.ravel(df_y)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print('Data pipeline complete.')

Data pipeline complete.


In [9]:
from sklearn.linear_model import SGDClassifier
clf = make_pipeline(StandardScaler(),
                    SGDClassifier(max_iter=1000, tol=1e-3))
clf.fit(X_train, y_train)
print('Training complete!')

Training complete!


In [10]:
y_pred = clf.predict(X_test)
# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.65


In [13]:
print("Your turn!")
year = 2023
qtr = input("What Quarter is it? ")
min = input("How many minutes are left in the quarter? ")
sec = input("How many seconds are left? ")
down = input("What down is it? ")
ytg = input("How many yards to a first down? ")
yl = input("Where is the ball (yds from the goal line, 0-100)? ")


team_dict = {"ARI": 1, "ATL": 2, "BAL": 3, "BUF": 4, "CAR": 5, "CHI": 6,
             "CHI": 6, "CIN": 7, "CLE": 8, "DAL": 9, "DEN": 10, "DET": 11,
             "GB": 12, "HOU": 13, "IND": 14, "JAX": 15, "KC": 16, "LV": 17,
             "LAC": 18, "LA": 19, "MIA": 20, "MIN": 21, "NE": 22, "NO": 23,
             "NYG": 24, "NYJ": 25, "PHI": 26, "PIT": 27, "SF": 28, "SEA": 29,
             "TB": 30, "TEN": 31, "WAS": 32, "OAK": 17, "SD": 18, "STL": 19
             }

offense_text = input("What team is on offense? (ex: LAC, TB) ")
offense = team_dict[offense_text]
defense_text = input("What team is on defense? (ex: SF, TEN) ")
defense = team_dict[defense_text]

print("Thinking...")
play_prediction = clf.predict([[year, qtr, min, sec, down, ytg, yl, offense, defense]])
if play_prediction == 1:
  play_prediction_string = "Rush"
elif play_prediction == 2:
  play_prediction_string = "Pass"
elif play_prediction == 3:
  play_prediction_string = "Special"
elif play_prediction == 4:
  play_prediction_string = "No Play"

print(f"Prediction: {offense_text} {play_prediction_string}")

Your turn!
What Quarter is it? 1
How many minutes are left in the quarter? 5
How many seconds are left? 1
What down is it? 1
How many yards to a first down? 5
Where is the ball (yds from the goal line, 0-100)? 51
What team is on offense? (ex: LAC, TB) TEN
What team is on defense? (ex: SF, TEN) SF
Thinking...
Prediction: TEN Pass


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
